## Example: Evaluating GLM Model with OpenAI Agents

This notebook demonstrates how to evaluate the GLM Model integrated with the OpenAI Agents framework. We will implement a model wrapper that adheres to the `agents.models.interface.Model` interface, specifically the `get_response` and `stream_response` methods.

In [3]:
import os
import base64

 
# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-962190cc-b2bd-47c0-b752-8de287a2a5c1" 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-4e9959c3-0935-4142-b789-734beb81d15a" 
os.environ["LANGFUSE_HOST"] = "http://localhost:3000"
 
# Build Basic Auth header.
LANGFUSE_AUTH = base64.b64encode(
    f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
).decode()
 
# Configure OpenTelemetry endpoint & headers
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_HOST") + "/api/public/otel"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"
 
# Your openai key
# os.environ["OPEN_AI_URL"] = "https://open.bigmodel.cn/api/paas/v4/"
os.environ["OPENAI_API_KEY"] = "f0c1fb9f5c534e55a66d9e539916fdb0.GQKa6HaX6MpT9ioJ"
os.environ["OPENAI_BASE_URL"] = "https://open.bigmodel.cn/api/paas/v4"

In [4]:
from langfuse import get_client
 
langfuse = get_client()

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
import logfire

# Configure logfire instrumentation
logfire.configure(
    service_name="my_agent_service",
    send_to_logfire=False,
)

# The method automatically patches the OpenAI Agents SDK to send logs via OTLP to Langfuse.
logfire.instrument_openai_agents()

Overriding of current TracerProvider is not allowed


In [9]:
import asyncio
from agents import Agent, Runner, RunConfig
from glm_agent_wrapper.models.glm_model import GLMModel

# Initialize the GLM Model
glm_model = GLMModel(model_name="GLM-4.5")

class CustomAgent(Agent):
    async def run(self, task: str):
        response = await glm_model.get_response(task)
        return response

async def main():
    agent = CustomAgent(
        name="GLM Assistant",
        instructions="You are a helpful assistant using the GLM model."
    )
    result = await Runner.run(
        agent,
        "Explain the importance of evaluating AI agents.",
        run_config=RunConfig(model=glm_model)
    )
    print(result.final_output)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

langfuse.flush()

16:03:23.696 OpenAI Agents trace: Agent workflow
16:03:23.697   Agent run: 'GLM Assistant'
16:03:23.700     Responses API with 'gpt-4o'


Error getting response: Error code: 404 - {'timestamp': '2025-08-20T08:03:25.869+00:00', 'status': 404, 'error': 'Not Found', 'path': '/v4/responses'}. (request_id: None)


NotFoundError: Error code: 404 - {'timestamp': '2025-08-20T08:03:25.869+00:00', 'status': 404, 'error': 'Not Found', 'path': '/v4/responses'}

In [11]:
glm_model = GLMModel(model_name="GLM-4.5")

glm_model.get_response("What is the capital of France?")

'The capital of France is **Paris**.  \n\nHere are a few key details:  \n- **Location**: Situated in the north-central part of France, along the Seine River.  \n- **Significance**: Paris is not only the political capital but also the country’s largest city, a global hub for art, fashion, gastronomy, and culture.  \n- **Landmarks**: Home to iconic sites like the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and the Champs-Élysées.  \n\nParis has been the capital of France since at least the 10th century and remains the heart of French governance, economy, and cultural identity.'

### Summary
In this notebook, we have set up a custom agent that utilizes the GLM model for generating responses. The `GLMModel` class implements the required methods from the `agents.models.interface.Model` interface, allowing for seamless integration with the agent framework.